# 🎓 Knowledge Tracing - Demo Interativo

Este notebook demonstra o sistema completo de Knowledge Tracing.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from dkt_model import load_model
from recommender import recommend_next

print('✅ Imports carregados')

## 1. Carregar Dados

In [ ]:
df = pd.read_csv('data/real_combined_dataset.csv')
print(f'Dataset: {len(df)} interações, {df["student_id"].nunique()} alunos')
df.head()

## 2. EDA - Análise Exploratória

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Taxa de acerto por fonte
df.groupby('source')['correct'].mean().plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Taxa de Acerto por Dataset')
axes[0].set_ylabel('Taxa de Acerto')

# Distribuição de interações por aluno
df.groupby('student_id').size().hist(bins=30, ax=axes[1], color='salmon')
axes[1].set_title('Distribuição de Interações por Aluno')
axes[1].set_xlabel('Número de Interações')

plt.tight_layout()
plt.show()

## 3. Carregar Modelo Treinado

In [ ]:
model, metadata = load_model('models/dkt.pt')
item_to_idx = metadata.get('item_to_idx', {})
print(f'✅ Modelo carregado: {sum(p.numel() for p in model.parameters())} parâmetros')
print(f'Val AUC: {metadata.get("val_auc", "N/A")}')

## 4. Demo de Recomendação

In [ ]:
# Selecionar 3 alunos com diferentes níveis
students = df['student_id'].unique()[:3]

for student_id in students:
    student_df = df[df['student_id'] == student_id].sort_values('timestamp')
    history = student_df.to_dict('records')[:10]  # Primeiras 10 interações
    
    candidates = list(set(df['item_id'].unique()) - set(student_df['item_id'].unique()))[:5]
    
    rec = recommend_next(history, model, candidates, item_to_idx, strategy='target')
    
    print(f'\n📚 Aluno: {student_id[:8]}...')
    print(f'   Taxa de acerto: {student_df["correct"].mean():.2%}')
    print(f'   Recomendação: {rec["item_id"]} (P={rec["p_estimated"]:.2f})')
    print(f'   Justificativa: {rec["rationale"]}')

## 5. Visualizar Evolução de P(correct)

In [ ]:
# Simular evolução de P(correct) ao longo do tempo
student_id = students[0]
student_df = df[df['student_id'] == student_id].sort_values('timestamp')

# Aqui você implementaria a predição sequencial
# Para simplificar, vamos plotar a taxa de acerto móvel

student_df['correct_ma'] = student_df['correct'].rolling(window=5, min_periods=1).mean()

plt.figure(figsize=(10, 4))
plt.plot(student_df['correct_ma'], marker='o', label='Taxa de Acerto (MA-5)')
plt.axhline(y=0.7, color='r', linestyle='--', label='Target (0.7)')
plt.xlabel('Interação')
plt.ylabel('P(correct)')
plt.title(f'Evolução do Aluno {student_id[:8]}...')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## ✅ Conclusão

Este notebook demonstrou:
- Carregamento e exploração de dados reais
- Uso do modelo DKT para predições
- Recomendações personalizadas para diferentes perfis
- Visualização da evolução do aluno

**Próximos passos**: Experimente diferentes estratégias e compare resultados!